In [2]:
import pandas as pd
import datapackage
import time
import json
import geopy.distance
import time

In [3]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

### Read json into dataframe

In [ ]:
df = pd.read_json('data/ufo_awesome.json',lines=True)

In [4]:
df.head()

,description,duration,location,reported_at,shape,sighted_at
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009
1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,,19951010
2,Telephoned Report:CA woman visiting daughter w...,,"Shelton, WA",19950103,,19950101
3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,,19950510
4,Anonymous caller repts. sighting 4 ufo&apos;s ...,,"Seattle, WA",19950614,,19950611


### Get lat/long of location & convert epoch time to date in original data

#### Example of how to use geopy

In [10]:
from geopy.geocoders import Nominatim

In [11]:
geolocator = Nominatim()

In [12]:
geolocator.geocode("Milwaukee, WI")

Location((43.0349931, -87.922497, 0.0))

#### since Sachin already got all the location long/lat we read in the cached file now

In [13]:
location_dict = json.load(open("cached_location/location_cache.txt"))

In [14]:
location_dict[' Iowa City, IA']

[41.6612561, -91.5299106, u'Iowa City', u'Iowa', u'us']

#### Add new columns to original data

In [15]:
# temporarily set the following columns to NA for easy error lookup later
df['reported_at_date'] = "NA"
df['sighted_at_date'] = "NA"
df['geocoded_latitude'] = "NA"
df['geocoded_longitude'] = "NA"

In [16]:
df.head(2)

,description,duration,location,reported_at,shape,sighted_at,reported_at_date,sighted_at_date,geocoded_latitude,geocoded_longitude
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,NA,NA,NA,NA
1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,,19951010,NA,NA,NA,NA


#### Go through each row in original data

In [ ]:
# read from cache
read_cache = True

for index,row in df.iterrows():
#     print row['location']
#     print row['reported_at']
#     print row['sighted_at']
    reported_at_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(row['reported_at']))
    df.ix[index, 'reported_at_date'] = reported_at_date
    
    sighted_at_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(row['sighted_at']))
    df.ix[index, 'sighted_at_date'] = sighted_at_date
    
    if not read_cache:
        geocoded_location = geolocator.geocode(row['location'])
        if geocoded_location:
            df.ix[index, 'geocoded_latitude'] = geocoded_location.latitude
            df.ix[index, 'geocoded_longitude'] = geocoded_location.longitude
    else:
        if row['location'] in location_dict:
            # [41.6612561, -91.5299106, u'Iowa City', u'Iowa', u'us']
            latitude,longitude,city,state,country = location_dict[row['location']]
            df.ix[index, 'geocoded_latitude'] = latitude
            df.ix[index, 'geocoded_longitude'] = longitude
#     print index

#### Pickling df now that each row has longitude/latitude

In [ ]:
# df.to_pickle('data/data_with_long_lat.pkl')

#### if you want to get the df from the pickle file

In [18]:
# df = pd.read_pickle('data/data_with_long_lat.pkl')

In [20]:
df.head(1)

,description,duration,location,reported_at,shape,sighted_at,reported_at_date,sighted_at_date,geocoded_latitude,geocoded_longitude
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,1970-08-19 14:56:49,1970-08-19 14:56:49,41.6613,-91.5299


## Join airport data with df

### Read in ONLY US airport codes

In [176]:
# https://datahub.io/core/airport-codes#pandas
data_url = 'https://datahub.io/core/airport-codes/datapackage.json'

package = datapackage.Package(data_url)

resources = package.resources
for resource in resources:
    if resource.tabular:
        airport_df = pd.read_csv(resource.descriptor['path'])
        airport_df = airport_df[airport_df.iso_country == "US"]

In [177]:
airport_df.head(5)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [178]:
airport_df.shape

(22389, 18)

#### unique airport types

In [179]:
airport_df.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

#### There are 172 large airports

In [181]:
airport_df[airport_df.type == 'large_airport'].shape

(172, 18)

In [182]:
large_airport_df = airport_df[airport_df.type == 'large_airport']

In [183]:
large_airport_df.shape

(172, 18)

In [184]:
large_airport_df.head(3)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
25647,16091,KABQ,large_airport,Albuquerque International Sunport Airport,35.040199,-106.609001,5355.0,NaN,US,US-NM,Albuquerque,yes,KABQ,ABQ,ABQ,http://www.abqsunport.com/,http://en.wikipedia.org/wiki/Albuquerque_Inter...,NaN
25666,3364,KADW,large_airport,Andrews Air Force Base,38.810799,-76.866997,280.0,NaN,US,US-MD,Camp Springs,no,KADW,ADW,ADW,http://www.jba.af.mil/,http://en.wikipedia.org/wiki/Joint_Base_Andrews,NaN
25679,3366,KAFW,large_airport,Fort Worth Alliance Airport,32.987598,-97.318802,722.0,NaN,US,US-TX,Fort Worth,no,KAFW,AFW,AFW,http://www.allianceairport.com/,http://en.wikipedia.org/wiki/Fort_Worth_Allian...,NaN


#### 687 medium airports

In [186]:
airport_df[airport_df.type == 'medium_airport'].shape

(687, 18)

In [187]:
medium_airport_df = airport_df[airport_df.type == 'medium_airport']

In [188]:
medium_airport_df.shape

(687, 18)

#### 13865 small airports

In [190]:
airport_df[airport_df.type == 'small_airport'].shape

(13865, 18)

In [191]:
small_airport_df = airport_df[airport_df.type == 'small_airport']

In [192]:
small_airport_df.shape

(13865, 18)

#### 858 closed airports -- could be interesting to see if there is any correlation here

In [193]:
airport_df[airport_df.type == 'closed'].shape

(858, 18)

In [206]:
closed_airport_df = airport_df[airport_df.type == 'closed']

In [207]:
closed_airport_df.shape

(858, 18)

#### NOT USING: distance function that takes in 2 coordinates --> returns distance in miles

In [39]:
def distance(c1, c2):
    # great_circle is less accurate than vincenty but is faster
    return geopy.distance.great_circle(c1, c2).miles

In [40]:
distance((41.6613,-91.5299), (41.680318, -91.598565))

3.780708989414044

### haversine distance 

In [47]:
# https://stackoverflow.com/questions/43577086/pandas-calculate-haversine-distance-within-each-group-of-rows/43577275#43577275
# https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas/29546836#29546836
import numpy as np

# haversine distance
def distance_np(c1, c2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1 = c1 
    lon2, lat2 = c2
    
    lon2_,lat2_ = lon2,lat2
    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    miles = km/1.609344
    return miles

In [48]:
large_airport_lat_long_df = airport_df[airport_df.type == 'large_airport'][['latitude_deg', 'longitude_deg']]

In [ ]:
distance_np((41.6613,-91.5299),(large_airport_lat_long_df['latitude_deg'],large_airport_lat_long_df['longitude_deg'])).tolist()

In [46]:
min(distance_np((41.6613,-91.5299),(large_airport_lat_long_df['latitude_deg'],large_airport_lat_long_df['longitude_deg'])).tolist())

12.49873655937495

#### differences between different distance metrics
It is clear that vincenty is most precise, but haversine is great for quick computation

In [50]:
# great circle distance
distance((41.6613,-91.5299), (41.680318, -91.598565))

3.780708989414044

In [51]:
distance_np((41.6613,-91.5299), (41.680318, -91.598565))

4.7414498663059712

In [52]:
geopy.distance.vincenty((41.6613,-91.5299), (41.680318, -91.598565)).miles

3.787775939330663

--------------------

### NOT USING: function to get closest latitude and longitude given a list of lat/long and a target lat/long

In [ ]:
def closest(data, v):
    return min(data, key=lambda p: distance((v['latitude_deg'],v['longitude_deg']),(p['latitude_deg'],p['longitude_deg'])))

In [ ]:
tempDataList = [{'latitude_deg': 39.7612992, 'longitude_deg': -86.1519681}, 
                {'latitude_deg': 39.762241,  'longitude_deg': -86.158436 }, 
                {'latitude_deg': 39.7622292, 'longitude_deg': -86.1578917}]

v = {'latitude_deg': 39.7622290, 'longitude_deg': -86.1519750}

In [ ]:
closest(tempDataList, v)

--------------

## add distance&name to closest large airport

In [214]:
# maintain a copy of the df before adding new columns
df = pd.read_pickle('data/data_with_long_lat.pkl')

#### remove all rows without a coordinate

In [215]:
df = df[df.geocoded_latitude != "NA"]

----------

In [216]:
df['closest_LARGE_airport_name'] = ''
df['closest_LARGE_airport_distance'] = float("inf")

df['closest_MEDIUM_airport_name'] = ''
df['closest_MEDIUM_airport_distance'] = float("inf")

df['closest_SMALL_airport_name'] = ''
df['closest_SMALL_airport_distance'] = float("inf")

In [217]:
df.head(1)

,description,duration,location,reported_at,shape,sighted_at,reported_at_date,sighted_at_date,geocoded_latitude,geocoded_longitude,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,1970-08-19 14:56:49,1970-08-19 14:56:49,41.6613,-91.5299,,inf,,inf,,inf


#### The following takes ~53 minutes to run -- read the resulting df from pickle (data/df_nearest_airports.pkl) instead of running

In [ ]:
t0 = time.time()

large_airport_df = airport_df[airport_df.type == 'large_airport']
large_airport_df['distance_from_target'] = float("inf")

medium_airport_df = airport_df[airport_df.type == 'medium_airport']
medium_airport_df['distance_from_target'] = float("inf")

small_airport_df = airport_df[airport_df.type == 'small_airport']
small_airport_df['distance_from_target'] = float("inf")

closed_airport_df = airport_df[airport_df.type == 'closed']
closed_airport_df['distance_from_target'] = float("inf")

for index,row in df.iterrows():
    longitude = row['geocoded_longitude']
    latitude = row['geocoded_latitude']

    # skip rows without coordinates
    if longitude == "NA" or latitude == "NA":
        continue
    
    large_airport_with_distance_df = large_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(large_airport_df['latitude_deg'],large_airport_df['longitude_deg'])))
    smallest_distance_row = large_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_LARGE_airport_name = smallest_distance_row['name']
    closest_LARGE_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_LARGE_airport_name'] = closest_LARGE_airport_name
    df.ix[index, 'closest_LARGE_airport_distance'] = closest_LARGE_airport_distance
    
    medium_airport_with_distance_df = medium_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(medium_airport_df['latitude_deg'],medium_airport_df['longitude_deg'])))
    smallest_distance_row = medium_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_MEDIUM_airport_name = smallest_distance_row['name']
    closest_MEDIUM_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_MEDIUM_airport_name'] = closest_MEDIUM_airport_name
    df.ix[index, 'closest_MEDIUM_airport_distance'] = closest_MEDIUM_airport_distance
    
    small_airport_with_distance_df = small_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(small_airport_df['latitude_deg'],small_airport_df['longitude_deg'])))
    smallest_distance_row = small_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_SMALL_airport_name = smallest_distance_row['name']
    closest_SMALL_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_SMALL_airport_name'] = closest_SMALL_airport_name
    df.ix[index, 'closest_SMALL_airport_distance'] = closest_SMALL_airport_distance
    
    closed_airport_with_distance_df = closed_airport_df.assign(distance_from_target = distance_np((latitude,longitude),(closed_airport_df['latitude_deg'],closed_airport_df['longitude_deg'])))
    smallest_distance_row = closed_airport_with_distance_df.nsmallest(1, 'distance_from_target').head(1).squeeze()
    closest_CLOSED_airport_name = smallest_distance_row['name']
    closest_CLOSED_airport_distance = smallest_distance_row['distance_from_target']
    df.ix[index, 'closest_CLOSED_airport_name'] = closest_CLOSED_airport_name
    df.ix[index, 'closest_CLOSED_airport_distance'] = closest_CLOSED_airport_distance
    
    print index
t1 = time.time()
print t1-t0

In [230]:
df.head(5)

,description,duration,location,reported_at,shape,sighted_at,reported_at_date,sighted_at_date,geocoded_latitude,geocoded_longitude,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance,closest_CLOSED_airport_name,closest_CLOSED_airport_distance
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,1970-08-19 14:56:49,1970-08-19 14:56:49,41.6613,-91.5299,The Eastern Iowa Airport,12.498008,Chippewa Valley Regional Airport,6.616011,Marion Airport,0.683006,Pete's Patch Airport,3.912630
1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,,19951010,1970-08-19 14:56:51,1970-08-19 14:56:50,43.035,-87.9225,General Mitchell International Airport,1.801887,Kenosha Regional Airport,1.157715,Herbert C. Maas Airport,1.716202,General Mitchell Intl Heliport,1.552094
2,Telephoned Report:CA woman visiting daughter w...,,"Shelton, WA",19950103,,19950101,1970-08-19 14:41:43,1970-08-19 14:41:41,47.2151,-123.101,McChord Air Force Base,43.233065,Olympia Regional Airport,16.484896,Sanderson Field,3.339607,Teufel's Farm Strip,64.820703
3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,,19950510,1970-08-19 14:48:30,1970-08-19 14:48:30,38.9519,-92.3337,Bill & Hillary Clinton National Airport/Adams ...,13.834501,Columbia Regional Airport,7.889876,Willhite Airport,3.120229,Iowa Army Natl Guard Heliport,11.244884
4,Anonymous caller repts. sighting 4 ufo&apos;s ...,,"Seattle, WA",19950614,,19950611,1970-08-19 14:50:14,1970-08-19 14:50:11,47.6038,-122.33,Boeing Field King County International Airport,3.343976,Snohomish County (Paine Field) Airport,11.644950,Renton Municipal Airport,8.871067,Blaine Municipal Airport,58.478816


#### save df to pickle

In [44]:
df.to_pickle('data/df_nearest_airports.pkl')

In [45]:
df.to_csv('data/df_nearest_airports.csv',encoding='utf-8')

In [16]:
df = pd.read_pickle('data/df_nearest_airports.pkl')

-------

In [30]:
df.head()

,description,duration,location,reported_at,shape,sighted_at,reported_at_date,sighted_at_date,geocoded_latitude,geocoded_longitude,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance,closest_CLOSED_airport_name,closest_CLOSED_airport_distance,closest_airport_distance,closest_airport_name
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,1970-08-19 14:56:49,1970-08-19 14:56:49,41.6613,-91.5299,The Eastern Iowa Airport,12.498008,Chippewa Valley Regional Airport,6.616011,Marion Airport,0.683006,Pete's Patch Airport,3.912630,0.683006,Marion Airport
1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,,19951010,1970-08-19 14:56:51,1970-08-19 14:56:50,43.035,-87.9225,General Mitchell International Airport,1.801887,Kenosha Regional Airport,1.157715,Herbert C. Maas Airport,1.716202,General Mitchell Intl Heliport,1.552094,1.157715,Kenosha Regional Airport
2,Telephoned Report:CA woman visiting daughter w...,,"Shelton, WA",19950103,,19950101,1970-08-19 14:41:43,1970-08-19 14:41:41,47.2151,-123.101,McChord Air Force Base,43.233065,Olympia Regional Airport,16.484896,Sanderson Field,3.339607,Teufel's Farm Strip,64.820703,3.339607,Sanderson Field
3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,,19950510,1970-08-19 14:48:30,1970-08-19 14:48:30,38.9519,-92.3337,Bill & Hillary Clinton National Airport/Adams ...,13.834501,Columbia Regional Airport,7.889876,Willhite Airport,3.120229,Iowa Army Natl Guard Heliport,11.244884,3.120229,Willhite Airport
4,Anonymous caller repts. sighting 4 ufo&apos;s ...,,"Seattle, WA",19950614,,19950611,1970-08-19 14:50:14,1970-08-19 14:50:11,47.6038,-122.33,Boeing Field King County International Airport,3.343976,Snohomish County (Paine Field) Airport,11.644950,Renton Municipal Airport,8.871067,Blaine Municipal Airport,58.478816,3.343976,Boeing Field King County International Airport


In [18]:
df['closest_airport_distance'] = df.loc[:, ['closest_LARGE_airport_distance', 'closest_MEDIUM_airport_distance', 'closest_SMALL_airport_distance']].min(axis=1)

In [26]:
df['closest_airport_name'] = ""
for index,row in df.iterrows():
    closest_airport = row['closest_airport_distance']
    
    if closest_airport == row['closest_LARGE_airport_distance']:
        df.ix[index, 'closest_airport_name'] = row['closest_LARGE_airport_name']
    elif closest_airport == row['closest_MEDIUM_airport_distance']:
        df.ix[index, 'closest_airport_name'] = row['closest_MEDIUM_airport_name']
    else:
        df.ix[index, 'closest_airport_name'] = row['closest_SMALL_airport_name']

------

In [47]:
location_dict = json.load(open("cached_location/location_cache.txt"))

location_dict[' Iowa City, IA']

[41.6612561, -91.5299106, u'Iowa City', u'Iowa', u'us']

#### add city and state as columns

In [48]:
df = pd.read_pickle('data/df_nearest_airports.pkl')

In [50]:
df.head(1)

,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance,closest_CLOSED_airport_name,closest_CLOSED_airport_distance,closest_airport_distance,closest_airport_name
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,41.6613,-91.5299,The Eastern Iowa Airport,12.498008,Chippewa Valley Regional Airport,6.616011,Marion Airport,0.683006,Pete's Patch Airport,3.91263,0.683006,Marion Airport


In [51]:
df['city'] = ""
df['state'] = ""
for index,row in df.iterrows():
    location = row['location']
    lat,lon,city,state,country = location_dict[location]
    df.ix[index, 'city'] = city
    df.ix[index, 'state'] = state

In [52]:
df.head()

,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,closest_LARGE_airport_name,closest_LARGE_airport_distance,closest_MEDIUM_airport_name,closest_MEDIUM_airport_distance,closest_SMALL_airport_name,closest_SMALL_airport_distance,closest_CLOSED_airport_name,closest_CLOSED_airport_distance,closest_airport_distance,closest_airport_name,city,state
0,"Man repts. witnessing &quot;flash, followed by...",,"Iowa City, IA",19951009,,19951009,41.6613,-91.5299,The Eastern Iowa Airport,12.498008,Chippewa Valley Regional Airport,6.616011,Marion Airport,0.683006,Pete's Patch Airport,3.912630,0.683006,Marion Airport,Iowa City,Iowa
1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,,19951010,43.035,-87.9225,General Mitchell International Airport,1.801887,Kenosha Regional Airport,1.157715,Herbert C. Maas Airport,1.716202,General Mitchell Intl Heliport,1.552094,1.157715,Kenosha Regional Airport,Milwaukee,Wisconsin
2,Telephoned Report:CA woman visiting daughter w...,,"Shelton, WA",19950103,,19950101,47.2151,-123.101,McChord Air Force Base,43.233065,Olympia Regional Airport,16.484896,Sanderson Field,3.339607,Teufel's Farm Strip,64.820703,3.339607,Sanderson Field,Shelton,Washington
3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,,19950510,38.9519,-92.3337,Bill & Hillary Clinton National Airport/Adams ...,13.834501,Columbia Regional Airport,7.889876,Willhite Airport,3.120229,Iowa Army Natl Guard Heliport,11.244884,3.120229,Willhite Airport,Columbia,Missouri
4,Anonymous caller repts. sighting 4 ufo&apos;s ...,,"Seattle, WA",19950614,,19950611,47.6038,-122.33,Boeing Field King County International Airport,3.343976,Snohomish County (Paine Field) Airport,11.644950,Renton Municipal Airport,8.871067,Blaine Municipal Airport,58.478816,3.343976,Boeing Field King County International Airport,Seattle,Washington
